# Performance analysis
### The notebook compares the performance of the different modalities and the different chunk sizes. The information extracted here is presented in the _Results_ chapter in the second (_Waveform-based and spike-timing features allow near-perfect classification of PYR and PV cells_) and fifth (_Features based exclusively on spatial information allow accurate classification of PYR and PV cells_) subsections, as well as in  in _Figure 2_ and _Figure 5_.
#### The insights from this notebook shold be updated in the three parameters of constants.py: BEST_WF_CHUNK , BEST_ST_CHUNK, and BEST_SPATIAL_CHUNK.

In [1]:
from notebooks_constants import SRC_PATH
import sys
sys.path.insert(0, SRC_PATH)

import pandas as pd
import os
import numpy as np
import scipy.stats as stats

from paths import MAIN_RES

pd.options.display.max_columns = 500
pd.options.display.max_rows = 500

In [2]:
PATH = MAIN_RES

In [3]:
results = pd.read_csv(PATH, index_col=0)
results.chunk_size = pd.to_numeric(results.chunk_size, downcast='integer')
chunk_sizes = results.chunk_size.unique()

complete = results.dropna(how='all', axis=1)
grouped_complete = complete.groupby(by=['modality', 'chunk_size'])
grouped_complete[['auc']].median()

auc
modality     chunk_size          
spatial      0           0.827883
             25          0.962995
             50          0.945640
             100         0.943488
             200         0.923982
             400         0.931010
             800         0.903041
             1600        0.906770
spike-timing 0           0.975043
             25          0.975043
             50          0.973465
             100         0.967011
             200         0.969880
             400         0.978199
             800         0.975760
             1600        0.977338
waveform     0           0.995410
             25          0.999139
             50          0.999426
             100         0.999426
             200         0.999426
             400         0.999426
             800         0.999713
             1600        0.998566

## Waveform

In [4]:
wf_res = complete[complete.modality == 'waveform']
grouped_wf = wf_res.groupby(by=['chunk_size'])
grouped_wf[['auc']].quantile(q=[0.25, 0.5, 0.75])

auc
chunk_size               
0          0.25  0.978199
           0.50  0.995410
           0.75  1.000000
25         0.25  0.974039
           0.50  0.999139
           0.75  1.000000
50         0.25  0.986517
           0.50  0.999426
           0.75  1.000000
100        0.25  0.971888
           0.50  0.999426
           0.75  1.000000
200        0.25  0.973537
           0.50  0.999426
           0.75  1.000000
400        0.25  0.972174
           0.50  0.999426
           0.75  1.000000
800        0.25  0.972533
           0.50  0.999713
           0.75  1.000000
1600       0.25  0.969449
           0.50  0.998566
           0.75  1.000000

In [5]:
wf_aucs = [wf_res.auc[wf_res.chunk_size == cs] for cs in chunk_sizes]
wf_aucs_0 = wf_aucs[0]
print(f"comparing CS=0 to baseline for WF models:", stats.wilcoxon(wf_aucs_0-0.5, alternative='greater'))
for cs_aucs, cs in zip(wf_aucs[1:], chunk_sizes[1:]):
    print(f"comparing CS=0 to CS={cs} for WF models:", stats.wilcoxon(cs_aucs, wf_aucs_0, alternative='greater'))
    ri = cs_aucs.to_numpy() / wf_aucs_0.to_numpy()
    print(f"The median relative improvement is: {np.median(ri)}")

comparing CS=0 to baseline for WF models: WilcoxonResult(statistic=1275.0, pvalue=3.130562367449191e-10)
comparing CS=0 to CS=25 for WF models: WilcoxonResult(statistic=401.0, pvalue=0.015649703398377783)
The median relative improvement is: 1.0
comparing CS=0 to CS=50 for WF models: WilcoxonResult(statistic=424.0, pvalue=0.0013853640884081617)
The median relative improvement is: 1.0011490953719466
comparing CS=0 to CS=100 for WF models: WilcoxonResult(statistic=364.5, pvalue=0.06665615323960604)
The median relative improvement is: 1.0
comparing CS=0 to CS=200 for WF models: WilcoxonResult(statistic=413.5, pvalue=0.008730637193996126)
The median relative improvement is: 1.0002870264064292
comparing CS=0 to CS=400 for WF models: WilcoxonResult(statistic=321.5, pvalue=0.141113494887741)
The median relative improvement is: 1.0
comparing CS=0 to CS=800 for WF models: WilcoxonResult(statistic=362.0, pvalue=0.07264069940082778)
The median relative improvement is: 1.0
comparing CS=0 to CS=1600

Largest relative improvement was found for 50-spikes chunks, improvement:

In [6]:
temp = complete[complete.modality == 'waveform']
temp_0 = temp[temp.chunk_size == 0].auc.to_numpy()
temp_50 = temp[temp.chunk_size == 50].auc.to_numpy()
diff = 100 * (temp_50 - temp_0) / temp_0
print(f"Q25, Q50 and Q75  for chunk size = 50: {np.quantile(diff, q=[0.25, 0.5, 0.75])}%")

Q25, Q50 and Q75  for chunk size = 50: [0.         0.11490954 0.55336064]%


## Spike-timing

In [7]:
temp_res = complete[complete.modality == 'spike-timing']
grouped_temp = temp_res.groupby(by=['chunk_size'])
grouped_temp[['auc']].quantile(q=[0.25, 0.5, 0.75])

auc
chunk_size               
0          0.25  0.961059
           0.50  0.975043
           0.75  0.986661
25         0.25  0.959983
           0.50  0.975043
           0.75  0.981641
50         0.25  0.960413
           0.50  0.973465
           0.75  0.980493
100        0.25  0.949297
           0.50  0.967011
           0.75  0.983864
200        0.25  0.955321
           0.50  0.969880
           0.75  0.982788
400        0.25  0.961704
           0.50  0.978199
           0.75  0.985513
800        0.25  0.962278
           0.50  0.975760
           0.75  0.985298
1600       0.25  0.965003
           0.50  0.977338
           0.75  0.987378

In [8]:
temp_aucs = [temp_res.auc[temp_res.chunk_size == cs] for cs in chunk_sizes]
temp_aucs_0 = temp_aucs[0]
print(f"comparing CS=0 to baseline for Temporal models:", stats.wilcoxon(temp_aucs_0 - 0.5, alternative='greater'))
for cs_aucs, cs in zip(temp_aucs[1:], chunk_sizes[1:]):
    print(f"comparing CS=0 to CS={cs} for Spatial models:", stats.wilcoxon(cs_aucs, temp_aucs_0, alternative='greater'))
    ri = cs_aucs.to_numpy() / temp_aucs_0.to_numpy()
    print(f"The effect size is: {np.median(ri)}")

comparing CS=0 to baseline for Temporal models: WilcoxonResult(statistic=1275.0, pvalue=3.7750486097697307e-10)
comparing CS=0 to CS=25 for Spatial models: WilcoxonResult(statistic=347.0, pvalue=0.997479675102897)
The effect size is: 0.9948261491643633
comparing CS=0 to CS=50 for Spatial models: WilcoxonResult(statistic=354.5, pvalue=0.9948634279635458)
The effect size is: 0.9959098022721157
comparing CS=0 to CS=100 for Spatial models: WilcoxonResult(statistic=210.0, pvalue=0.999981611594078)
The effect size is: 0.9925021612654501
comparing CS=0 to CS=200 for Spatial models: WilcoxonResult(statistic=291.5, pvalue=0.998821913896245)
The effect size is: 0.994857009561388
comparing CS=0 to CS=400 for Spatial models: WilcoxonResult(statistic=671.5, pvalue=0.37137010488299904)
The effect size is: 1.0008792684264534
comparing CS=0 to CS=800 for Spatial models: WilcoxonResult(statistic=577.0, pvalue=0.6380073927260191)
The effect size is: 1.0002912111475017
comparing CS=0 to CS=1600 for Spati

Largest relative improvement was found for 1600-spikes chunks, improvement:

In [9]:
temp = complete[complete.modality == 'spike-timing']
temp_0 = temp[temp.chunk_size == 0].auc.to_numpy()
temp_1600 = temp[temp.chunk_size == 1600].auc.to_numpy()
diff = 100 * (temp_1600 - temp_0) / temp_0
print(f"Q25, Q50 and Q75  for chunk size = 1600: {np.quantile(diff, q=[0.25, 0.5, 0.75])}%")

Q25, Q50 and Q75  for chunk size = 1600: [-0.3928307   0.27547579  0.74311007]%


## Spatial

In [10]:
spat_res = complete[complete.modality == 'spatial']
grouped_spat = spat_res.groupby(by=['chunk_size'])
grouped_spat[['auc']].quantile(q=[0.25, 0.5, 0.75])

auc
chunk_size               
0          0.25  0.802065
           0.50  0.827883
           0.75  0.848609
25         0.25  0.949369
           0.50  0.962995
           0.75  0.974900
50         0.25  0.931727
           0.50  0.945640
           0.75  0.960843
100        0.25  0.927711
           0.50  0.943488
           0.75  0.955250
200        0.25  0.898881
           0.50  0.923982
           0.75  0.946357
400        0.25  0.899742
           0.50  0.931010
           0.75  0.947935
800        0.25  0.865175
           0.50  0.903041
           0.75  0.928141
1600       0.25  0.870697
           0.50  0.906770
           0.75  0.927352

In [11]:
spat_aucs = [spat_res.auc[spat_res.chunk_size == cs] for cs in chunk_sizes]
spat_aucs_0 = spat_aucs[0]
print(f"comparing CS=0 to baseline for Spatial models:", stats.wilcoxon(spat_aucs_0-0.5, alternative='greater'))
for cs_aucs, cs in zip(spat_aucs[1:], chunk_sizes[1:]):
    print(f"comparing CS=0 to CS={cs} for Spatial models:", stats.wilcoxon(cs_aucs, spat_aucs_0, alternative='greater'))
    ri = cs_aucs.to_numpy() / spat_aucs_0.to_numpy()
    print(f"The effect size is: {np.median(ri)}")

comparing CS=0 to baseline for Spatial models: WilcoxonResult(statistic=1275.0, pvalue=3.7759023791403374e-10)
comparing CS=0 to CS=25 for Spatial models: WilcoxonResult(statistic=1275.0, pvalue=3.7776104381903367e-10)
The effect size is: 1.163251630387498
comparing CS=0 to CS=50 for Spatial models: WilcoxonResult(statistic=1275.0, pvalue=3.778464727931783e-10)
The effect size is: 1.1488755210325325
comparing CS=0 to CS=100 for Spatial models: WilcoxonResult(statistic=1275.0, pvalue=3.778464727931783e-10)
The effect size is: 1.1385109151311457
comparing CS=0 to CS=200 for Spatial models: WilcoxonResult(statistic=1275.0, pvalue=3.7776104381903367e-10)
The effect size is: 1.106771359149762
comparing CS=0 to CS=400 for Spatial models: WilcoxonResult(statistic=1275.0, pvalue=3.7776104381903367e-10)
The effect size is: 1.1185841571399644
comparing CS=0 to CS=800 for Spatial models: WilcoxonResult(statistic=1252.5, pvalue=1.4531857366560612e-09)
The effect size is: 1.0927681673621708
compari

Largest relative improvement was found for 25-spikes chunks, improvement:

In [12]:
temp = complete[complete.modality == 'spatial']
temp_0 = temp[temp.chunk_size == 0].auc.to_numpy()
temp_25 = temp[temp.chunk_size == 25].auc.to_numpy()
diff = 100 * (temp_25 - temp_0) / temp_0
print(f"Q25, Q50 and Q75  for chunk size = 25: {np.quantile(diff, q=[0.25, 0.5, 0.75])}%")

Q25, Q50 and Q75  for chunk size = 25: [13.27175918 16.32516304 19.63062307]%


## Cross-comparisons

In [13]:
st_1600 = complete[complete.chunk_size == 1600]
st_1600 = st_1600[st_1600.modality == 'spike-timing'].auc

wf_50 = complete[complete.chunk_size == 50]
wf_50 = wf_50[wf_50.modality == 'waveform'].auc

spat_25 = complete[complete.chunk_size == 25]
spat_25 = spat_25[spat_25.modality == 'spatial'].auc

print('comparing waveform and spatial:', stats.wilcoxon(wf_50, spat_25))
print('comparing spike-timing and spatial:', stats.wilcoxon(spat_25, st_1600))

comparing waveform and spatial: WilcoxonResult(statistic=10.0, pvalue=1.3791775344591993e-09)
comparing spike-timing and spatial: WilcoxonResult(statistic=159.5, pvalue=6.599637581622721e-06)
